# Assessment Graphs
This notebook is for creating graphs based on how modules are assessed and how susceptible they appear to ChatGPT. All module information was collected from publicly available module information on UCD modules in the College of Engineering and Architecture, the method of which can be seen in the Module Scraper Code notebook.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import re
import pathlib as Path
import html5lib
import json

In [2]:
def file_finder(school=None, level=None, givenList=None):
    dir_raw=Path.Path("ModuleInformation")
    
    subdirectory=""
    #Save the file in its desired format
    if level != None:
        subdirectory+= "/Level=%d" %(level)
        
    if school != None:
        subdirectory+= "_School="+school.replace(" ", "-")
        
    if givenList != None:
        subdirectory += "SelectedModules"
        
        
    if len(subdirectory) > 0:
        dir_raw =dir_raw / subdirectory
        

    with open(dir_raw / "assessments.json", 'r') as infile:
        print("Reading from %s" % dir_raw)
        assessments=pd.read_json(infile)
    with open(dir_raw / "descriptors.json", 'r') as infile:
        print("Reading from %s" % dir_raw)
        descriptors=pd.read_json(infile)
    
    return assessments, descriptors

In [3]:
assessments, descriptors = file_finder()

assessments

Reading from ModuleInformation
Reading from ModuleInformation


,Description,Timing,Open Book Exam,Component Scale,Must Pass Component,% of Final Grade,Assessment Type,Scaled % of Final Grade
0,Examination: 2 HOUR Examination,2 hour End of Trimester Exam,No,Alternative linear conversion grade scale 40%,No,65,Examination,65
1,"Lab Report: Two in-person labs will be run , w...",Varies over the Trimester,None,Alternative linear conversion grade scale 40%,No,10,Lab Report,10
2,Continuous Assessment: 3 Online quizzes: 1) ci...,Varies over the Trimester,None,Alternative linear conversion grade scale 40%,No,25,Continuous Assessment,25
3,Lab Report: Laboratories,Varies over the Trimester,None,Graded,No,20,Lab Report,20
4,Examination: Second in-class examination,Week 8,Yes,Standard conversion grade scale 40%,No,15,Examination,15
...,...,...,...,...,...,...,...,...
89,Continuous Assessment: Brightspace quiz,Week 3,None,Other,No,10,Continuous Assessment,10
90,Continuous Assessment: Brightspace quiz,Week 7,None,Other,No,20,Continuous Assessment,20
91,Class Test: In-semester class test #2 (online ...,Unspecified,None,Standard conversion grade scale 40%,No,35,Class Test,35
92,Group Project: A group project assignment whic...,Varies over the Trimester,None,Standard conversion grade scale 40%,No,30,Group Project,30


In [4]:
susceptibility={"Assignment" :"susceptible", \
                "Attendence": "not susceptible", \
                "Class Test" : "not susceptible", \
                "Continuous Assessment": "susceptible", \
               "Essay": "susceptible", \
                "Examination": "not susceptible", \
                "Fieldwork": "not susceptible", \
                "Group Project": "susceptible", \
                "Journal": "not susceptible",\
               "Lab Report": "not susceptible", \
                "Multiple Choice Questionnaire (MCQ)": "not susceptible", \
                "Oral Examination": "not susceptible", \
               "Portfolio" : "susceptible",  \
                "Practical Examination": "not susceptible", \
                "Presentation" : "not susceptible", \
                "Project": "susceptible", \
               "Seminar": "not susceptible", \
               "Studio Examination" : "not susceptible"}

SyntaxError: invalid syntax (2846357085.py, line 1)